In [1]:
#https://stackoverflow.com/questions/69025750/how-to-fine-tune-huggingface-bert-model-for-text-classification

In [2]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('./../data/datasets/aclImdb/train')
test_texts, test_labels = read_imdb_split('./../data/datasets/aclImdb/test')

<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_716/627383030.py:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  labels.append(0 if label_dir is "neg" else 1)


In [3]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [32]:
len(val_texts)

5000

In [24]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased', max_length=100)

In [25]:
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [49]:
from transformers import BertTokenizer, TFBertForSequenceClassification, TFDistilBertModel, TFDistilBertForSequenceClassification
transformer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', output_hidden_states=True, num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_133', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use 

In [50]:
import tensorflow as tf
input_ids = tf.keras.Input(shape=(768, ),dtype='int32', name='input_ids')
attention_mask = tf.keras.Input(shape=(768, ), dtype='int32', name='attention_mask')

transformer = transformer_model([input_ids, attention_mask])    
hidden_states = transformer[1] # get output_hidden_states

hidden_states_size = 4 # count of the last states 
hiddes_states_ind = list(range(-hidden_states_size, 0, 1))

selected_hiddes_states = tf.keras.layers.concatenate(tuple([hidden_states[i] for i in hiddes_states_ind]))

In [51]:
model = tf.keras.models.Model(inputs = [input_ids, attention_mask], outputs = selected_hiddes_states)

In [52]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 768)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 768)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_for_sequence_cla TFSequenceClassifier 66955010    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 768, 3072)    0           tf_distil_bert_for_sequence

In [53]:
def tokenize(sentences, max_length=768, padding='max_length'):
    """Tokenize using the Huggingface tokenizer
    Args:
        sentences: String or list of string to tokenize
        padding: Padding method ['do_not_pad'|'longest'|'max_length']
    """
    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf"
    )

In [54]:
X = dict(tokenize(val_texts[:2]))

In [55]:
x = model(X)

In [56]:
x.shape

TensorShape([2, 768, 3072])